In [1]:
import pandas as pd
import requests
import json
import os

tokenCanvas = "10747~lGBNJqxBXPlYOYvb8lHvCNn9yXQo91Gu6LC3vfySqhrVl0F2hjzKNyD8KBVOsGKL"
parametreCanvas = {'per_page': '100'}
hodeCanvas = {'Authorization': 'Bearer ' + tokenCanvas}


def finn_rel(link_header):
    link_header_dict = {}
    for link in link_header.split(","):
        url, rel = link.strip().split(";")
        rel = rel.split('=')[1]
        link_header_dict[rel.strip().replace('"', '')] = url.strip().replace('<', '').replace('>', '')      # Kommentar: denne er med BERRE for å skape balanse i .org-fila: \"
    return link_header_dict



In [2]:
emne = [29605, 28031, 29547, 29622, 28438, 29571, 30477, 28627, 29552, 29686, 29679]
# emne= [29605]

In [5]:
student_liste = []
tilsett_liste = []
for e in emne:
    url_namn = f"https://hvl.instructure.com/api/v1/courses/{e}"
    respons_namn = requests.get(url_namn, headers=hodeCanvas, params=parametreCanvas)
    emnenamn = respons_namn.json()['name']
    url = f"https://hvl.instructure.com/api/v1/courses/{e}/enrollments"
    respons = requests.get(url, headers=hodeCanvas, params=parametreCanvas)
    print(f"Hentar personar frå {e}: ", end="")
    dr_liste_studentar = []
    dr_liste_tilsette = []
    hentmeir = True
    while hentmeir:
        if 200 <= respons.status_code < 300:
            data = respons.json()
            hentmeir = "next" in respons.headers['link']
            if hentmeir:
                print(respons.headers['link'])
                url = finn_rel(respons.headers['link'])['next']
                # print(url)   # for kontrollen sin skuld
    # Hent ut data (NB! dette vil variere frå endepunkt til endepunkt)
            dataliste_studentar = []
            dataliste_tilsette = []
            for element in data:
                if element['type']=="StudentEnrollment":
                    dataliste_studentar.append([element['course_id'], emnenamn, element['sis_course_id'], element['user']['login_id'], element['user']['name']])
                else:
                    dataliste_tilsette.append([element['course_id'], emnenamn, element['sis_course_id'], element['user']['login_id'], element['user']['name']])
            df_studentar = pd.DataFrame(dataliste_studentar, columns=['course_id', 'emnenamn', 'sis_course_id', 'login_id', 'name'])
            df_tilsette = pd.DataFrame(dataliste_tilsette, columns=['course_id', 'emnenamn', 'sis_course_id', 'login_id', 'name'])
            dr_liste_studentar.append(df_studentar)
            dr_liste_tilsette.append(df_tilsette)
    # og etter at eg er ferdig å lese inn slår eg dei saman:
    alledata_studentar = pd.concat((df for df in dr_liste_studentar if not df.empty), ignore_index=True)
    alledata_tilsette = pd.concat((df for df in dr_liste_tilsette if not df.empty), ignore_index=True)
    student_liste.append(alledata_studentar)
    tilsett_liste.append(alledata_tilsette)
    print(len(alledata_studentar), len(alledata_tilsette))
namneliste_studentar = pd.concat((df for df in student_liste if not df.empty), ignore_index=True)
namneliste_tilsette = pd.concat((df for df in tilsett_liste if not df.empty), ignore_index=True)
namneliste_tilsette.drop_duplicates(subset=['course_id', 'login_id'], inplace=True, ignore_index=True)
namneliste_studentar.drop_duplicates(subset=['course_id', 'login_id'], inplace=True, ignore_index=True)
namneliste_studentar.to_csv("studentar.csv", index=False)
namneliste_tilsette.to_csv("tilsette.csv", index=False)

Hentar personar frå 29605: 16 7
Hentar personar frå 28031: 76 24
Hentar personar frå 29547: 59 14
Hentar personar frå 29622: <https://hvl.instructure.com/api/v1/courses/29622/enrollments?page=first&per_page=100>; rel="current",<https://hvl.instructure.com/api/v1/courses/29622/enrollments?page=bookmark:WyJUZWFjaGVyRW5yb2xsbWVudCIsIkhvcGUsIENocmlzdGluZSIsMTMwMTAwN10&per_page=100>; rel="next",<https://hvl.instructure.com/api/v1/courses/29622/enrollments?page=first&per_page=100>; rel="first"
<https://hvl.instructure.com/api/v1/courses/29622/enrollments?page=first&per_page=100>; rel="current",<https://hvl.instructure.com/api/v1/courses/29622/enrollments?page=bookmark:WyJUZWFjaGVyRW5yb2xsbWVudCIsIkhvcGUsIENocmlzdGluZSIsMTMwMTAwN10&per_page=100>; rel="next",<https://hvl.instructure.com/api/v1/courses/29622/enrollments?page=first&per_page=100>; rel="first"
<https://hvl.instructure.com/api/v1/courses/29622/enrollments?page=first&per_page=100>; rel="current",<https://hvl.instructure.com/api/v1/c

KeyboardInterrupt: 